# Plan Semilla 3.0 - Artificial Intelligence Module

# State Space Search

In this challenge you will implement some search algorithms for navigation problems.

Your task is to create an autonomous vehicle that goes from point A to point B. You will be given some grid configurations that will simulate aerial images.

The symbols that form the grid have a special meaning as they specify the type of the terrain and the cost to enter a grid cell with that type of terrain:


In [2]:
import numpy as np

print('Symbol', '   Meaning       ', 'Cost')
print('🛣', '       Street        ', 1)
print('🌲', '       Forest        ', 10)
print('🚸', '       School street ', 2)
print('🚦', '       Traffic light ', 3)
print('🚔', '       Police        ', 4)
print('🚘', '       Traffic       ', 6)
print('🛑', '       Stop sign     ', 5)
print('🚧', '       Construction  ', 7)
print('🏢', '       Buildings     ', 15)

COSTS = {'🛣': 1, '🌲': 10, '🚸': 2, '🚦': 3, '🚔': 4, '🚘': 6, '🛑': 5, '🚧': 7, '🏢': 15}

Symbol    Meaning        Cost
🛣        Street         1
🌲        Forest         10
🚸        School street  2
🚦        Traffic light  3
🚔        Police         4
🚘        Traffic        6
🛑        Stop sign      5
🚧        Construction   7
🏢        Buildings      15


In [3]:
test_map_1 = np.array([['🛣', '🛣', '🛣', '🛣'],
                       ['🛣', '🌲', '🌲', '🛣'],
                       ['🛣', '🌲', '🌲', '🛣'],
                       ['🛣', '🛣', '🛣', '🛣'],
                       ['🌲', '🛣', '🛣', '🛣']])

map1 = np.array([['🛣', '🚸', '🛣', '🛣', '🛣', '🛣', '🛣', '🛑', '🛣', '🛣', '🚦', '🛣', '🛣', '🛣', '🚔', '🛣'],
                 ['🛣', '🌲', '🌲', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚘', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣'],
                 ['🛣', '🛣', '🛣', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚘', '🛣', '🌲', '🌲', '🛣', '🚘', '🛣'],
                 ['🛣', '🛑', '🛣', '🚸', '🛣', '🌲', '🛣', '🛣', '🏢', '🛣', '🛣', '🌲', '🌲', '🛣', '🛣', '🚦'],
                 ['🌲', '🚘', '🚘', '🛣', '🛣', '🛣', '🚘', '🛣', '🏢', '🛣', '🛣', '🌲', '🛣', '🛑', '🛣', '🛣'],
                 ['🛣', '🛣', '🛣', '🏢', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚦', '🛣', '🛣', '🛣', '🛣', '🚸'],
                 ['🛣', '🛣', '🏢', '🏢', '🏢', '🛣', '🛑', '🛣', '🛣', '🛣', '🚘', '🛣', '🛣', '🚸', '🛣', '🛣'],
                 ['🚦', '🛣', '🏢', '🚔', '🛣', '🛣', '🏢', '🚧', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚔'],
                 ['🛣', '🛣', '🛣', '🚔', '🛣', '🛣', '🏢', '🏢', '🏢', '🚸', '🛣', '🛑', '🚘', '🚔', '🛣', '🛣'],
                 ['🛣', '🛑', '🛣', '🛣', '🛣', '🛣', '🏢', '🏢', '🏢', '🛣', '🛣', '🏢', '🛣', '🛣', '🛣', '🚘'],
                 ['🛣', '🛣', '🛣', '🌲', '🌲', '🚦', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🛣', '🚧', '🚘'],
                 ['🚘', '🚸', '🛣', '🌲', '🌲', '🛣', '🛣', '🛣', '🛣', '🚔', '🛣', '🛣', '🚦', '🚘', '🛣', '🛣'],
                 ['🚘', '🛣', '🚧', '🛣', '🛣', '🛣', '🛑', '🛣', '🚘', '🚘', '🛣', '🚘', '🛣', '🛣', '🛣', '🛣']])

In [3]:


class Agent:

    def __init__(self, world, costs, initial_state, goal_state):

        self.moves = np.array([[0,1], [0,-1], [-1,0], [1,0]])
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.current_state = initial_state
        self.world = world
        self.costs = costs

    def possible_moves(self):

        succ_states, succ_actions, succ_costs = [],[],[]

        for move in self.moves:

            # check that the next state is in the range of the world
            #print(self.current_state)
            if 0 <= self.current_state[0] + move[0] < self.world.shape[0] and 0 <= self.current_state[1] + move[1] < self.world.shape[1]:

                # move and assign new_state to the position after moving
                new_state = [self.current_state[0] + move[0], self.current_state[1] + move[1]]

                action = [move[0], move[1]]

                cost = self.costs[self.world[new_state[0], new_state[1]]]

                succ_states.append(new_state)
                succ_actions.append(action)
                succ_costs.append(cost)

        return succ_states, succ_actions, succ_costs


    def set_current_state(self, nex_state):

        self.current_state = nex_state



class Node:

    def __init__(self, state, parent=None, action=None, cost=None):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        #self.cost = cost

        self.depth = 0
        self.path_cost = 0

        if parent:
            self.path_cost = parent.path_cost + cost
            self.depth = parent.depth + 1


    def successors(self, agent):

        succ_states, succ_actions, succ_costs = agent.possible_moves()
        children = []

        for i in range(len(succ_states)):

            children.append(Node(succ_states[i], self, succ_actions[i], succ_costs[i]))
        return  children

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        # Team init        
        while node:
            path_back.append(node)
            node = node.parent 
        # Team final
        return list(reversed(path_back))


def depth_first_graph_search(agent):
    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # Stack
    explored = []
    cont=0
    while frontier:
        node = frontier.pop() # LIFO
        if node.state == agent.goal_state:
            return node.path()
        cont=cont+1
        if(cont==40):
            break
        explored.append(node)
        print("\nNode: ",node)
        print("\ninitial_state",agent.initial_state)
        print("\nCosto: ",agent.costs)
        print("\ncurrent_state: ",node.state)
        print("\ngoal_state",agent.goal_state)
        agent.current_state=node.state
        for child in node.successors(agent):
            if child not in explored and child not in frontier:
                frontier.append(child)
    return None


def breadth_first_graph_search(agent):
    """
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [Node(agent.initial_state)]  # Queue

    explored = []
    while frontier:
        node = frontier.pop(0)
        if node.state == agent.goal_state:
            return node.path()
        explored.append(node.state)

        for child in node.successors(agent):
            if child.state not in explored and child not in frontier:
                frontier.append(child)

    return None



def uniform_cost_search(agent):
    """
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(agent.initial_state))]  # PriorityQueue

    explored = []
    while frontier:
        node = frontier.pop(0)
        if agent.goal_state == node.state:
            return node.path()

        if node.state not in explored:
            explored.append(node.state)
            children = node.successors(agent)

            for child in children:
                index = 0
                for i in range(len(frontier)):
                    if child.path_cost < frontier[i].path_cost:
                        index = i + 1
                    else:
                        break
                frontier.insert(index, child)

    return None



def order_frontier(frontier):

    costs = []
    new_list = []

    for i in range(len(frontier)):

        costs.append(frontier[i].path_cost)

    costs = np.array(costs)

    indexes = np.argsort(costs)

    for i in range(len(frontier)):

        new_list.append(frontier[indexes[i]])

    return list(reversed(new_list))


"""

Ada = Agent(map1, COSTS, [0,0], [12,15])
g_node = uniform_cost_search(Ada)

print(g_node.state)

p = g_node.path()

moves = {'01':'⏩','0-1':'⏪','-10':'⏫', '10':'⏬'}

map_copy = map1.copy()


for i in range(len(p)-1):

    ac = p[i]
    n = p[i+1]

    map_copy[ac.state[0], ac.state[1]] = moves[str(n.action[0])+str(n.action[1])]


for i in range(map_copy.shape[0]):
    print(map_copy[i,:])

"""

"\n\nAda = Agent(map1, COSTS, [0,0], [12,15])\ng_node = uniform_cost_search(Ada)\n\nprint(g_node.state)\n\np = g_node.path()\n\nmoves = {'01':'⏩','0-1':'⏪','-10':'⏫', '10':'⏬'}\n\nmap_copy = map1.copy()\n\n\nfor i in range(len(p)-1):\n\n    ac = p[i]\n    n = p[i+1]\n\n    map_copy[ac.state[0], ac.state[1]] = moves[str(n.action[0])+str(n.action[1])]\n\n\nfor i in range(map_copy.shape[0]):\n    print(map_copy[i,:])\n\n"

In [4]:
import numpy as np

world = np.array([[0, 1, 3],
                  [2, 3, 2],
                  [0, 1, 3]])

costs = {0: 10, 1: 1, 2: 5, 3:100}

initial_state = (0, 0)
goal_state = (2,2)

agent1 = Agent(world, costs, initial_state, goal_state)

"""
print(agent1.world)
print(agent1.costs)
print(agent1.initial_state)
print(agent1.goal_state)
print(agent1.moves)
print(agent1.possible_moves())
"""

path = depth_first_graph_search(agent1)



Node:  <__main__.Node object at 0x000001DC4DDB58B0>

initial_state (0, 0)

Costo:  {0: 10, 1: 1, 2: 5, 3: 100}

current_state:  (0, 0)

goal_state (2, 2)

Node:  <__main__.Node object at 0x000001DC4DE21A30>

initial_state (0, 0)

Costo:  {0: 10, 1: 1, 2: 5, 3: 100}

current_state:  [1, 0]

goal_state (2, 2)

Node:  <__main__.Node object at 0x000001DC4DE218E0>

initial_state (0, 0)

Costo:  {0: 10, 1: 1, 2: 5, 3: 100}

current_state:  [2, 0]

goal_state (2, 2)

Node:  <__main__.Node object at 0x000001DC4DE214C0>

initial_state (0, 0)

Costo:  {0: 10, 1: 1, 2: 5, 3: 100}

current_state:  [1, 0]

goal_state (2, 2)

Node:  <__main__.Node object at 0x000001DC4F244EB0>

initial_state (0, 0)

Costo:  {0: 10, 1: 1, 2: 5, 3: 100}

current_state:  [2, 0]

goal_state (2, 2)

Node:  <__main__.Node object at 0x000001DC4F244B50>

initial_state (0, 0)

Costo:  {0: 10, 1: 1, 2: 5, 3: 100}

current_state:  [1, 0]

goal_state (2, 2)

Node:  <__main__.Node object at 0x000001DC4F244FA0>

initial_state (0